In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, SGDRegressor

# ----------------- Load and Prepare Data ---------------------
data = pd.read_csv("D:\YOUTUBE MODEL\CLIMATE PREDECTION\climate_prediction.csv")
data = data.drop(columns='Unnamed: 0')
x = data[['Humidity (%)', 'Wind Speed (km/h)', 'Pressure (hPa)', 'Cloud Cover (%)']]
y = data['Temperature (°C)']

# ----------------- Split Data ---------------------
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

# ----------------- Standardize Features ---------------------
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# -----------grid search parameter--------------------
param_grid_lasso = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}
param_grid_ridge = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}
param_grid_elasticnet = {'alpha': [0.001, 0.01, 0.1, 1, 10], 'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]}
param_grid_sgd = {'alpha': [0.0001, 0.001, 0.01, 0.1], 'penalty': ['l2', 'l1', 'elasticnet'], 'max_iter': [1000, 2000]}

# -------------------- key (model name) :  value(model function() , para_grid) -------------------
models = {
    'Linear Regression': LinearRegression(),
    'Lasso': (Lasso(), param_grid_lasso),
    'Ridge': (Ridge(), param_grid_ridge),
    'ElasticNet': (ElasticNet(), param_grid_elasticnet),
    'SGDRegressor': (SGDRegressor(), param_grid_sgd)
}

# --------empty dictionary---------------------
best_models = {}

# ----------linear model training ----------------------------------------
model_1 = LinearRegression()
model_1.fit(x_train, y_train)
mp1 = model_1.predict(x_train)              #-------mp1 -model_predict_linear
mse_linear = mean_squared_error(y_train,mp1)
r2_linear = r2_score(y_train, mp1) * 100
best_models['Linear Regression'] = {'Best Score (MSE)': mse_linear, 'Accuracy (R2 Score)': r2_linear}

# ------------------grid search for other model -----------------------
for model_name, model_info in models.items(): #-------------models contain ( key : values)
    if model_name  == 'Linear Regression': #(model_name= key)
        continue  # Skip as it's already trained
    
    model, param_grid = model_info #( value = model() ,  para grid)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
    grid_search.fit(x_train, y_train)


    # --------------  # Save best parameters and MSE for each model
    best_models[model_name] = {
        'Best Estimator': grid_search.best_estimator_,
        'Best Params': grid_search.best_params_,
        'Best Score (MSE)': -grid_search.best_score_,
        'Accuracy (R2 Score)': r2_score(y_train, grid_search.predict(x_train)) * 100
    }
for model_name, model_details in best_models.items():
    print(f"{model_name} - Best Params: {model_details.get('Best Params', 'N/A')}")
    print(f"{model_name} - Best MSE: {model_details['Best Score (MSE)']:.2f}")
    print(f"{model_name} - Accuracy (R2 Score): {model_details['Accuracy (R2 Score)']:.2f}%\n")

Linear Regression - Best Params: N/A
Linear Regression - Best MSE: 9.33
Linear Regression - Accuracy (R2 Score): 95.84%

Lasso - Best Params: {'alpha': 0.001}
Lasso - Best MSE: 9.36
Lasso - Accuracy (R2 Score): 95.84%

Ridge - Best Params: {'alpha': 0.001}
Ridge - Best MSE: 9.36
Ridge - Accuracy (R2 Score): 95.84%

ElasticNet - Best Params: {'alpha': 0.001, 'l1_ratio': 0.9}
ElasticNet - Best MSE: 9.36
ElasticNet - Accuracy (R2 Score): 95.84%

SGDRegressor - Best Params: {'alpha': 0.001, 'max_iter': 2000, 'penalty': 'l2'}
SGDRegressor - Best MSE: 9.35
SGDRegressor - Accuracy (R2 Score): 95.84%



In [ ]:
# ----------------- Predict Temperature Based on User Input ---------------------
print("\nPlease enter the following details to predict the Temperature:")

humidity = float(input("Enter Humidity (%): "))
wind_speed = float(input("Enter Wind Speed (km/h): "))
pressure = float(input("Enter Pressure (hPa): "))
cloud_cover = float(input("Enter Cloud Cover (%): "))

# Prepare user input and standardize
user_input = pd.DataFrame([[humidity, wind_speed, pressure, cloud_cover]], columns=x.columns)
user_input_scaled = scaler.transform(user_input)

# Predict temperature (using best model found, for example, Lasso)
best_model = best_models['Lasso']['Best Estimator']  # Example: Use Lasso's best estimator for prediction
predicted_temp = best_model.predict(user_input_scaled)


In [ ]:
data